# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 12 2022 10:51AM,241644,21,620459,"NBTY Global, Inc.",Released
1,Jul 12 2022 10:50AM,241643,21,619846,"NBTY Global, Inc.",Released
2,Jul 12 2022 10:42AM,241642,19,8265137,Else Nutrition,Released
3,Jul 12 2022 10:42AM,241641,10,8265099,MedStone Pharma LLC,Released
4,Jul 12 2022 10:42AM,241641,10,8265100,MedStone Pharma LLC,Released
5,Jul 12 2022 10:42AM,241641,10,8265101,MedStone Pharma LLC,Released
6,Jul 12 2022 10:42AM,241641,10,8265102,MedStone Pharma LLC,Released
7,Jul 12 2022 10:42AM,241641,10,8265103,MedStone Pharma LLC,Released
8,Jul 12 2022 10:42AM,241641,10,8265104,MedStone Pharma LLC,Released
9,Jul 12 2022 10:42AM,241641,10,8265105,MedStone Pharma LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,241640,Executing,1
36,241641,Released,10
37,241642,Released,1
38,241643,Released,1
39,241644,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241640,NaN,1.0,NaN
241641,NaN,NaN,10.0
241642,NaN,NaN,1.0
241643,NaN,NaN,1.0
241644,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241522,0.0,18.0,37.0
241523,0.0,0.0,63.0
241529,0.0,0.0,49.0
241560,0.0,13.0,9.0
241562,0.0,0.0,20.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241522,0,18,37
241523,0,0,63
241529,0,0,49
241560,0,13,9
241562,0,0,20


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241522,0,18,37
1,241523,0,0,63
2,241529,0,0,49
3,241560,0,13,9
4,241562,0,0,20


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241522,,18,37
1,241523,,,63
2,241529,,,49
3,241560,,13,9
4,241562,,,20


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 12 2022 10:51AM,241644,21,"NBTY Global, Inc."
1,Jul 12 2022 10:50AM,241643,21,"NBTY Global, Inc."
2,Jul 12 2022 10:42AM,241642,19,Else Nutrition
3,Jul 12 2022 10:42AM,241641,10,MedStone Pharma LLC
13,Jul 12 2022 10:41AM,241640,21,"NBTY Global, Inc."
14,Jul 12 2022 10:39AM,241639,10,Emerginnova
15,Jul 12 2022 10:01AM,241637,10,Emerginnova
16,Jul 12 2022 9:52AM,241636,20,Alumier Labs Inc.
17,Jul 12 2022 9:50AM,241635,10,ISDIN Corporation
28,Jul 12 2022 9:40AM,241633,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 12 2022 10:51AM,241644,21,"NBTY Global, Inc.",,,1
1,Jul 12 2022 10:50AM,241643,21,"NBTY Global, Inc.",,,1
2,Jul 12 2022 10:42AM,241642,19,Else Nutrition,,,1
3,Jul 12 2022 10:42AM,241641,10,MedStone Pharma LLC,,,10
4,Jul 12 2022 10:41AM,241640,21,"NBTY Global, Inc.",,1,
5,Jul 12 2022 10:39AM,241639,10,Emerginnova,,,1
6,Jul 12 2022 10:01AM,241637,10,Emerginnova,,,1
7,Jul 12 2022 9:52AM,241636,20,Alumier Labs Inc.,,,1
8,Jul 12 2022 9:50AM,241635,10,ISDIN Corporation,,,11
9,Jul 12 2022 9:40AM,241633,10,ISDIN Corporation,,,15


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 12 2022 10:51AM,241644,21,"NBTY Global, Inc.",1,,
1,Jul 12 2022 10:50AM,241643,21,"NBTY Global, Inc.",1,,
2,Jul 12 2022 10:42AM,241642,19,Else Nutrition,1,,
3,Jul 12 2022 10:42AM,241641,10,MedStone Pharma LLC,10,,
4,Jul 12 2022 10:41AM,241640,21,"NBTY Global, Inc.",,1,
5,Jul 12 2022 10:39AM,241639,10,Emerginnova,1,,
6,Jul 12 2022 10:01AM,241637,10,Emerginnova,1,,
7,Jul 12 2022 9:52AM,241636,20,Alumier Labs Inc.,1,,
8,Jul 12 2022 9:50AM,241635,10,ISDIN Corporation,11,,
9,Jul 12 2022 9:40AM,241633,10,ISDIN Corporation,15,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 12 2022 10:51AM,241644,21,"NBTY Global, Inc.",1,,
1,Jul 12 2022 10:50AM,241643,21,"NBTY Global, Inc.",1,,
2,Jul 12 2022 10:42AM,241642,19,Else Nutrition,1,,
3,Jul 12 2022 10:42AM,241641,10,MedStone Pharma LLC,10,,
4,Jul 12 2022 10:41AM,241640,21,"NBTY Global, Inc.",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 12 2022 10:51AM,241644,21,"NBTY Global, Inc.",1.0,NaN,NaN
1,Jul 12 2022 10:50AM,241643,21,"NBTY Global, Inc.",1.0,NaN,NaN
2,Jul 12 2022 10:42AM,241642,19,Else Nutrition,1.0,NaN,NaN
3,Jul 12 2022 10:42AM,241641,10,MedStone Pharma LLC,10.0,NaN,NaN
4,Jul 12 2022 10:41AM,241640,21,"NBTY Global, Inc.",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 12 2022 10:51AM,241644,21,"NBTY Global, Inc.",1.0,0.0,0.0
1,Jul 12 2022 10:50AM,241643,21,"NBTY Global, Inc.",1.0,0.0,0.0
2,Jul 12 2022 10:42AM,241642,19,Else Nutrition,1.0,0.0,0.0
3,Jul 12 2022 10:42AM,241641,10,MedStone Pharma LLC,10.0,0.0,0.0
4,Jul 12 2022 10:41AM,241640,21,"NBTY Global, Inc.",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4590494,452.0,19.0,1.0
16,241601,1.0,0.0,0.0
19,1208059,4.0,4.0,0.0
20,1208013,60.0,13.0,0.0
21,1208152,4.0,1.0,0.0
22,241623,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4590494,452.0,19.0,1.0
1,16,241601,1.0,0.0,0.0
2,19,1208059,4.0,4.0,0.0
3,20,1208013,60.0,13.0,0.0
4,21,1208152,4.0,1.0,0.0
5,22,241623,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,452.0,19.0,1.0
1,16,1.0,0.0,0.0
2,19,4.0,4.0,0.0
3,20,60.0,13.0,0.0
4,21,4.0,1.0,0.0
5,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,452.0
1,16,Released,1.0
2,19,Released,4.0
3,20,Released,60.0
4,21,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,21,22
Status,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,0.0
Executing,19.0,0.0,4.0,13.0,1.0,1.0
Released,452.0,1.0,4.0,60.0,4.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,21,22
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,19.0,0.0,4.0,13.0,1.0,1.0
2,Released,452.0,1.0,4.0,60.0,4.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,21,22
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,19.0,0.0,4.0,13.0,1.0,1.0
2,Released,452.0,1.0,4.0,60.0,4.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()